In [1]:
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

# Load the BERT tokenizer and model
model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def get_sentence_embedding(sentence):
    # Tokenize the sentence and get the input IDs
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)
    
    # Get the BERT embeddings (output)
    with torch.no_grad():
        outputs = model(**inputs)
    
    # The embeddings are in the last hidden state (batch_size, sequence_length, hidden_size)
    # We take the mean of the token embeddings (except [CLS] and [SEP])
    embeddings = outputs.last_hidden_state.mean(dim=1)
    
    return embeddings.squeeze().numpy()

def find_most_similar_sentence(input_sentence, paraphrase_candidates):
    # Get embedding for the input sentence
    input_embedding = get_sentence_embedding(input_sentence)
    
    # Get embeddings for all paraphrase candidates
    candidate_embeddings = [get_sentence_embedding(candidate) for candidate in paraphrase_candidates]
    
    # Compute cosine similarities between input sentence and paraphrase candidates
    similarities = [cosine_similarity([input_embedding], [candidate_embedding])[0][0] 
                    for candidate_embedding in candidate_embeddings]
    
    # Find the most similar sentence
    most_similar_index = similarities.index(max(similarities))
    
    return paraphrase_candidates[most_similar_index], similarities[most_similar_index]

# Example Vietnamese interpreted couplet
interpreted_couplet = "Trên trời cao ngàn sao sáng, dưới đất con người yên vui."

# Example list of paraphrase candidates
paraphrase_candidates = [
    "Bầu trời lấp lánh sao đêm, con người dưới đất an vui.",
    "Trên cao sao sáng rực rỡ, con người dưới đất hòa bình.",
    "Trời đầy sao sáng lấp lánh, con người dưới trần hạnh phúc.",
    "Sao trời chiếu sáng rực rỡ, nhân gian sống vui vẻ bình yên.",
    "Ngàn sao sáng soi đường, dưới trần thế nhân hòa an lạc."
]

# Find the most similar paraphrase
paraphrase, similarity = find_most_similar_sentence(interpreted_couplet, paraphrase_candidates)

print(f"Original Couplet: {interpreted_couplet}")
print(f"Best Paraphrase: {paraphrase}")
print(f"Similarity Score: {similarity}")


Original Couplet: Trên trời cao ngàn sao sáng, dưới đất con người yên vui.
Best Paraphrase: Bầu trời lấp lánh sao đêm, con người dưới đất an vui.
Similarity Score: 0.9070854783058167
